In [1]:
import os
#os.chdir("../")
os.getcwd()

In [11]:
import yaml 
from yaml.loader import SafeLoader
%load_ext autoreload
%autoreload 2
from run_training_pipeline import (
download_data,
prepare_data,
instantiate_dataset,
instantiate_model,
instantiate_loss,
instantiate_trainer,
instantiate_dataloader,
download_prepare_test
)
from train_pipeline_utils.download_data import load_donnees_test
from classes.data.satellite_image import SatelliteImage
from classes.data.labeled_satellite_image import SegmentationLabeledSatelliteImage
from utils.plot_utils import plot_list_segmentation_labeled_satellite_image
import torch

In [116]:
# length_delete = 75
# dir_path = "../data/PLEIADES/2022/MARTINIQUE/"

# files = os.listdir(dir_path)[:length_delete]

# for file in files:
#     file_path = os.path.join(dir_path, file)
#     if os.path.isfile(file_path):
#         os.remove(file_path)


In [3]:
with open("../config.yml") as f:
    config = yaml.load(f, Loader=SafeLoader)

list_data_dir = download_data(config)

In [4]:
list_output_dir = prepare_data(config, list_data_dir)
#download_prepare_test(config)
model = instantiate_model(config)


In [5]:
train_dl, valid_dl, test_dl = instantiate_dataloader(
    config, list_output_dir
)

In [6]:
import numpy as np

In [12]:
tile_size = config["donnees"]["tile size"]
batch_size = config["optim"]["batch size test"]
list_labeled_satellite_image = []

npatch = int((2000/tile_size)**2)
nbatchforfullimage = int(npatch/batch_size)

if not npatch % nbatchforfullimage == 0:
    print("Le nombre de patchs n'est pas divisible par la taille d'un batch")
    #return None 

for idx, batch in enumerate(test_dl):
    print("id_batch",idx)
    images, label, dic = next(iter(test_dl))
    output_model = model(images)
    mask_pred = np.array(torch.argmax(output_model, axis  = 1))

    for i in range(batch_size):    
        pthimg = dic["pathimage"][i]
        si = SatelliteImage.from_raster(
                file_path = pthimg,
                dep = None,
                date = None,
                n_bands= 3)
        si.normalize()
        
        list_labeled_satellite_image.append( 
            SegmentationLabeledSatelliteImage(
                satellite_image =  si ,
                label= mask_pred[i],
                source= "",
                labeling_date = ""
            )
        )
        
        del images, labels, dic
        
    if ((idx+1) % nbatchforfullimage) == 0:
        
        if not os.path.exists("img/"):
            os.makedirs("img/")
        
        fig1 = plot_list_segmentation_labeled_satellite_image(list_labeled_satellite_image,[0,1,2])
    
        filename = pthimg.split('/')[-1]
        filename = filename.split('.')[0]
        filename = '_'.join(filename.split('_')[0:6])
        plot_file = filename + ".png"
        
        fig1.savefig(plot_file)
        #mlflow.log_artifact(plot_file, artifact_path="plots")
            

    


In [152]:
images.shape

In [ ]:
#test pleiade
# test sentinel 2..
list_labeled_satellite_image = []

for idx, batch in enumerate(test_dl):
    images, labels, paths = batch
    model(images)